In [1]:
MEMORY_BACKEND = "graphiti"
RESET_MEMORY_BACKENDS = False

TOXIC_LATENCY = 0
TOXIC_BANDWIDTH = 0
TOXIC_SLOW_CLOSE = 0
TOXIC_TIMEOUT = 0
TOXIC_SLICER = 0
TOXIC_LIMIT_DATA = 0
TOXIC_RESET_PEER = 0

In [56]:
!make -C ../ down

make: Entering directory `c:/Users/bjpw/workspace/dmas-long-context-memory'
Stopping everything on ...
cd proxy && docker-compose --env-file ../.env down || echo ""
cd dmas && docker-compose --env-file ../.env down || echo ""
cd monitoring && docker-compose --env-file ../.env down || echo ""
docker network rm dmas-network 2>/dev/null || echo "Network removed"
"Network removed"
Done!
make: Leaving directory `c:/Users/bjpw/workspace/dmas-long-context-memory'


 Container ollama-proxy  Stopping
 Container openai-proxy  Stopping
 Container openai-proxy  Stopped
 Container openai-proxy  Removing
 Container openai-proxy  Removed
 Container ollama-proxy  Stopped
 Container ollama-proxy  Removing
 Container ollama-proxy  Removed
 Network dmas-network  Removing
 Network dmas-network  Resource is still in use
 Container coordinator  Stopping
 Container coordinator  Stopped
 Container coordinator  Removing
 Container coordinator  Removed
 Container locomo  Stopping
 Container responder  Stopping
 Container ollama  Stopping
 Container memory  Stopping
 Container responder  Stopped
 Container responder  Removing
 Container responder  Removed
 Container locomo  Stopped
 Container locomo  Removing
 Container locomo  Removed
 Container ollama  Stopped
 Container ollama  Removing
 Container ollama  Removed
 Container memory  Stopped
 Container memory  Removing
 Container memory  Removed
 Container neo4j  Stopping
 Container qdrant  Stopping
 Container qdra

In [57]:
if RESET_MEMORY_BACKENDS:
    !make -C ../ reset

In [2]:
import os
import re

def set_env_var(var_name: str, new_value: str, env_file: str = '../.env') -> None:
    env_file = os.path.abspath(env_file)
    content = ''
    if os.path.exists(env_file):
        with open(env_file, 'r') as f:
            content = f.read()
    pattern = re.compile(rf'^{re.escape(var_name)}=.*$', re.MULTILINE)
    if pattern.search(content):
        content = pattern.sub(f'{var_name}={new_value}', content)
    else:
        if content and not content.endswith('\n'):
            content += '\n'
        content += f'{var_name}={new_value}\n'
    with open(env_file, 'w') as f:
        f.write(content)

set_env_var('MEMORY_BACKEND', MEMORY_BACKEND)
set_env_var('TOXIC_LATENCY', str(TOXIC_LATENCY))
set_env_var('TOXIC_BANDWIDTH', str(TOXIC_BANDWIDTH))
set_env_var('TOXIC_SLOW_CLOSE', str(TOXIC_SLOW_CLOSE))
set_env_var('TOXIC_TIMEOUT', str(TOXIC_TIMEOUT))
set_env_var('TOXIC_SLICER', str(TOXIC_SLICER))
set_env_var('TOXIC_LIMIT_DATA', str(TOXIC_LIMIT_DATA))
set_env_var('TOXIC_RESET_PEER', str(TOXIC_RESET_PEER))

In [59]:
!make -C ../ build-monitoring

make: Entering directory `c:/Users/bjpw/workspace/dmas-long-context-memory'
Building monitoring services...
cd monitoring && docker-compose --env-file ../.env build --no-cache --pull
Monitoring build complete!
make: Leaving directory `c:/Users/bjpw/workspace/dmas-long-context-memory'


In [60]:
!make -C ../ build-proxy

make: Entering directory `c:/Users/bjpw/workspace/dmas-long-context-memory'
Building proxy services...
cd proxy && docker-compose --env-file ../.env build --no-cache --pull
#0 building with "desktop-linux" instance using docker driver

#1 [ollama-proxy internal] load build definition from Dockerfile
#1 transferring dockerfile: 281B 0.0s done
#1 DONE 0.0s

#2 [openai-proxy internal] load build definition from Dockerfile
#2 transferring dockerfile: 281B done
#2 DONE 0.0s

#3 [openai-proxy internal] load metadata for docker.io/library/python:3.11-slim
#3 DONE 0.5s

#4 [ollama-proxy internal] load .dockerignore
#4 transferring context: 2B done
#4 DONE 0.0s

#5 [openai-proxy internal] load .dockerignore
#5 transferring context: 2B done
#5 DONE 0.0s

#6 [ollama-proxy 1/6] FROM docker.io/library/python:3.11-slim@sha256:fa9b525a0be0c5ae5e6f2209f4be6fdc5a15a36fed0222144d98ac0d08f876d4
#6 resolve docker.io/library/python:3.11-slim@sha256:fa9b525a0be0c5ae5e6f2209f4be6fdc5a15a36fed0222144d98ac0d08

In [61]:
!make -C ../ build-dmas

make: Entering directory `c:/Users/bjpw/workspace/dmas-long-context-memory'
Building dmas services...
cd dmas && docker-compose --env-file ../.env build --no-cache --pull
#0 building with "desktop-linux" instance using docker driver

#1 [locomo internal] load build definition from Dockerfile
#1 transferring dockerfile: 294B done
#1 DONE 0.0s

#2 [responder internal] load build definition from Dockerfile
#2 transferring dockerfile: 294B done
#2 DONE 0.0s

#3 [memory internal] load build definition from Dockerfile
#3 transferring dockerfile: 294B done
#3 DONE 0.0s

#4 [memory internal] load metadata for docker.io/library/python:3.11-slim
#4 DONE 0.2s

#5 [locomo internal] load .dockerignore
#5 transferring context: 2B done
#5 DONE 0.0s

#6 [responder internal] load .dockerignore
#6 transferring context: 2B done
#6 DONE 0.0s

#7 [memory internal] load .dockerignore
#7 transferring context: 2B done
#7 DONE 0.0s

#8 [locomo 1/6] FROM docker.io/library/python:3.11-slim@sha256:fa9b525a0be0c5a

In [62]:
!make -C ../ up

make: Entering directory `c:/Users/bjpw/workspace/dmas-long-context-memory'
Setting DOCKER_GID=0
Starting services on ...
Docker GID: 0
docker network rm dmas-network 2>/dev/null || echo ""
""
cd proxy && docker-compose --env-file ../.env up -d
cd dmas && docker-compose --env-file ../.env up -d
cd monitoring && docker-compose --env-file ../.env up -d
All services started!
make: Leaving directory `c:/Users/bjpw/workspace/dmas-long-context-memory'


El sistema no puede encontrar la ruta especificada.
 Container ollama-proxy  Creating
 Container openai-proxy  Creating
 Container openai-proxy  Created
 Container ollama-proxy  Created
 Container ollama-proxy  Starting
 Container openai-proxy  Starting
 Container ollama-proxy  Started
 Container openai-proxy  Started
time="2025-11-06T09:22:00+01:00" level=warning msg="a network with name dmas-network exists but was not created for project \"dmas-long-context-memory\".\nSet `external: true` to use an existing network"
 Network dmas-long-context-memory_memory-internal  Creating
 Network dmas-long-context-memory_memory-internal  Created
 Container ollama  Creating
 Container locomo  Creating
 Container neo4j  Creating
 Container qdrant  Creating
 Container responder  Creating
 Container responder  Created
 Container ollama  Created
 Container neo4j  Created
 Container locomo  Created
 Container qdrant  Created
 Container memory  Creating
 Container memory  Created
 Container coordinator 

In [3]:
!docker exec memory printenv MEMORY_BACKEND

graphiti


In [4]:
conversation_index = 0
sessions = 19 # 1-19

In [5]:
import time
import json
import requests

PROM_URL = "http://localhost:9090/api/v1/query_range"
PROM_INSTANT_URL = "http://localhost:9090/api/v1/query"

def query_range(query, start, end, step="1s"):
    params = {"query": query, "start": start, "end": end, "step": step}
    r = requests.get(PROM_URL, params=params, timeout=10)
    r.raise_for_status()
    data = r.json()
    total = 0.0
    for result in data['data']['result']:
        for v in result['values']:
            total += float(v[1])
    return total

def get_instant_tokens(prometheus_url=PROM_INSTANT_URL):
    query = 'sum(openai_tokens_total{type="total"})'
    r = requests.get(prometheus_url, params={"query": query}, timeout=10)
    if r.status_code == 200:
        data = r.json()
        if data['status'] == 'success' and data['data']['result']:
            return float(data['data']['result'][0]['value'][1])
    return 0

def normalize_model_name(model_name):
    import re
    normalized = re.sub(r'-\d{4}-\d{2}-\d{2}$', '', model_name)
    return normalized

def get_openai_cost(start, end, pricing_file="openai_prices.json", tier="standard"):
    try:
        with open(pricing_file, 'r') as f:
            pricing_data = json.load(f)
        
        pricing_list = pricing_data['pricing']['text_tokens'][tier]
        text_pricing = {item['model']: {'input': item['input'], 'output': item['output']} for item in pricing_list}
        
        embedding_pricing = {item['model']: item['cost'] for item in pricing_data['pricing']['embeddings']}
        
        total_cost = 0.0
        
        for token_type in ['prompt', 'completion']:
            query = f'sum by (model) (openai_tokens_total{{type="{token_type}",endpoint="chat.completions"}})'
            
            r_before = requests.get(PROM_INSTANT_URL, params={"query": query, "time": start}, timeout=10)
            r_after = requests.get(PROM_INSTANT_URL, params={"query": query, "time": end + 2}, timeout=10)
            
            if r_before.status_code == 200 and r_after.status_code == 200:
                data_before = r_before.json()
                data_after = r_after.json()
                
                before_values = {}
                if data_before['status'] == 'success':
                    for result in data_before['data']['result']:
                        model = result['metric'].get('model', 'unknown')
                        before_values[model] = float(result['value'][1])
                
                if data_after['status'] == 'success':
                    for result in data_after['data']['result']:
                        model = result['metric'].get('model', 'unknown')
                        after_value = float(result['value'][1])
                        before_value = before_values.get(model, 0)
                        tokens = after_value - before_value
                        
                        if tokens > 0:
                            normalized_model = normalize_model_name(model)
                            
                            if model in text_pricing:
                                pricing_model = model
                            elif normalized_model in text_pricing:
                                pricing_model = normalized_model
                            else:
                                continue
                            
                            price_per_million = text_pricing[pricing_model]['input'] if token_type == 'prompt' else text_pricing[pricing_model]['output']
                            if price_per_million:
                                cost = (tokens / 1_000_000) * price_per_million
                                total_cost += cost
        
        query = f'sum by (model) (openai_tokens_total{{endpoint="embeddings"}})'
        
        r_before = requests.get(PROM_INSTANT_URL, params={"query": query, "time": start}, timeout=10)
        r_after = requests.get(PROM_INSTANT_URL, params={"query": query, "time": end + 2}, timeout=10)
        
        if r_before.status_code == 200 and r_after.status_code == 200:
            data_before = r_before.json()
            data_after = r_after.json()
            
            before_values = {}
            if data_before['status'] == 'success':
                for result in data_before['data']['result']:
                    model = result['metric'].get('model', 'unknown')
                    before_values[model] = float(result['value'][1])
            
            if data_after['status'] == 'success':
                for result in data_after['data']['result']:
                    model = result['metric'].get('model', 'unknown')
                    after_value = float(result['value'][1])
                    before_value = before_values.get(model, 0)
                    tokens = after_value - before_value
                    
                    if tokens > 0 and model in embedding_pricing:
                        cost = (tokens / 1_000_000) * embedding_pricing[model]
                        total_cost += cost
        
        return total_cost
    except Exception as e:
        print(f"Error calculating OpenAI cost: {e}")
        import traceback
        traceback.print_exc()
        return 0

def measure_function(func, *args, min_duration=15, **kwargs):
    tokens_before = get_instant_tokens()

    start = time.time()
    print(f"Starting function '{func.__name__}' at {start}")
    result = func(*args, **kwargs)
    end = time.time()
    print(f"Function '{func.__name__}' completed at {end}")
    actual_duration = end - start

    duration_s = max(int(actual_duration), min_duration)
    if actual_duration < min_duration:
        time.sleep(min_duration - actual_duration)
        end = time.time()

    time.sleep(2)
    
    tokens_after = get_instant_tokens()

    cpu_edge = query_range(f'sum(increase(docker_container_cpu_usage_total{{group="edge"}}[{duration_s}s]))', start, end)
    cpu_cloud = query_range(f'sum(increase(docker_container_cpu_usage_total{{group="cloud"}}[{duration_s}s]))', start, end)
    ram_edge = query_range(f'avg_over_time(docker_container_mem_usage{{group="edge"}}[{duration_s}s])', start, end)
    ram_cloud = query_range(f'avg_over_time(docker_container_mem_usage{{group="cloud"}}[{duration_s}s])', start, end)
    
    disk_edge_read = query_range(f'sum(increase(docker_container_blkio_io_service_bytes_recursive_read{{group="edge"}}[{duration_s}s]))', start, end)
    disk_edge_write = query_range(f'sum(increase(docker_container_blkio_io_service_bytes_recursive_write{{group="edge"}}[{duration_s}s]))', start, end)
    disk_edge = disk_edge_read + disk_edge_write
    
    disk_cloud_read = query_range(f'sum(increase(docker_container_blkio_io_service_bytes_recursive_read{{group="cloud"}}[{duration_s}s]))', start, end)
    disk_cloud_write = query_range(f'sum(increase(docker_container_blkio_io_service_bytes_recursive_write{{group="cloud"}}[{duration_s}s]))', start, end)
    disk_cloud = disk_cloud_read + disk_cloud_write
    
    network_edge = query_range(f'sum(increase(docker_container_net_rx_bytes{{group="edge"}}[{duration_s}s]))', start, end)
    network_cloud = query_range(f'sum(increase(docker_container_net_rx_bytes{{group="cloud"}}[{duration_s}s]))', start, end)
    
    tokens = int(tokens_after - tokens_before)
    
    token_cost = get_openai_cost(start, end)

    metrics = {
        "cpu_edge_ns": cpu_edge,
        "cpu_cloud_ns": cpu_cloud,
        "ram_edge_bytes": ram_edge,
        "ram_cloud_bytes": ram_cloud,
        "disk_edge_bytes": disk_edge,
        "disk_cloud_bytes": disk_cloud,
        "network_edge_bytes": network_edge,
        "network_cloud_bytes": network_cloud,
        "openai_tokens": tokens,
        "openai_cost_usd": token_cost,
        "api_latency_s": actual_duration,
        "metric_window_s": duration_s
    }

    return {"result": result, "metrics": metrics}

Optional: run `docker logs memory -f`

In [71]:
import shutil
import pandas as pd

def load_session(session_id):
    requests.post(f"http://localhost:8003/conversation/load/{conversation_index}/session/{session_id}")
    
memories = []

for i in range(1, sessions):
    print(f"Loading session: {i}")
    output = measure_function(load_session, i)
    row = {
        "memory": MEMORY_BACKEND,
        "time": time.time(),
        "conversation_index": conversation_index,
        "session_id": i,
        **output["metrics"],
        "toxic_latency": TOXIC_LATENCY,
        "toxic_bandwidth": TOXIC_BANDWIDTH,
        "toxic_slow_close": TOXIC_SLOW_CLOSE,
        "toxic_timeout": TOXIC_TIMEOUT,
        "toxic_slicer": TOXIC_SLICER,
        "toxic_limit_data": TOXIC_LIMIT_DATA,
        "toxic_reset_peer": TOXIC_RESET_PEER
    }
    print(row)
    memories.append(row)

Loading session: 1
Starting function 'load_session' at 1762418653.6927536
Function 'load_session' completed at 1762419057.9886968
{'memory': 'graphiti', 'time': 1762419060.4372354, 'conversation_index': 0, 'session_id': 1, 'cpu_edge_ns': 1353491370113.6267, 'cpu_cloud_ns': 6433879743163.855, 'ram_edge_bytes': 44140723040.99001, 'ram_cloud_bytes': 424490879748.3498, 'disk_edge_bytes': 82447955.92938206, 'disk_cloud_bytes': 340538563.99556214, 'network_edge_bytes': 43685359.84643223, 'network_cloud_bytes': 8086978368.682613, 'openai_tokens': 142569, 'openai_cost_usd': 7.52e-05, 'api_latency_s': 404.29594326019287, 'metric_window_s': 404, 'toxic_latency': 0, 'toxic_bandwidth': 0, 'toxic_slow_close': 0, 'toxic_timeout': 0, 'toxic_slicer': 0, 'toxic_limit_data': 0, 'toxic_reset_peer': 0}
Loading session: 2
Starting function 'load_session' at 1762419060.4674408
Function 'load_session' completed at 1762419623.410736
{'memory': 'graphiti', 'time': 1762419626.0870783, 'conversation_index': 0, '

In [72]:
if memories:
    df = pd.DataFrame(memories)
    output_dir = "results"
    os.makedirs(output_dir, exist_ok=True)
    
    is_constrained = (
        int(TOXIC_LATENCY) > 0 or
        int(TOXIC_BANDWIDTH) > 0 or
        int(TOXIC_SLOW_CLOSE) > 0 or
        int(TOXIC_TIMEOUT) > 0 or
        int(TOXIC_SLICER) > 0 or
        int(TOXIC_LIMIT_DATA) > 0 or
        int(TOXIC_RESET_PEER) > 0
    )
    
    if is_constrained:
        output_file = f"{MEMORY_BACKEND}_conv_{conversation_index}_sessions_constrained.csv"
    else:
        output_file = f"{MEMORY_BACKEND}_conv_{conversation_index}_sessions.csv"
    
    output_path = os.path.join(output_dir, output_file)
    
    if os.path.exists(output_path):
        if is_constrained:
            backup_path = os.path.join(output_dir, f"{MEMORY_BACKEND}_conv_{conversation_index}_sessions_constrained_backup.csv")
        else:
            backup_path = os.path.join(output_dir, f"{MEMORY_BACKEND}_conv_{conversation_index}_sessions_backup.csv")
        shutil.move(output_path, backup_path)
        print(f"Existing file backed up to {backup_path}")
    
    df.to_csv(output_path, index=False)
    print(f"Saved new results to {output_path}")

Existing file backed up to results\graphiti_conv_0_sessions_backup.csv
Saved new results to results\graphiti_conv_0_sessions.csv


In [6]:
import requests

questions = requests.get(f"http://localhost:8002/conversations/index/{conversation_index}/questions").json()
print(questions)

{'conversation_index': 0, 'sample_id': 'conv-26', 'total_questions': 199, 'questions': [{'sample_id': 'conv-26', 'question_id': 'conv-26_q_0', 'question': 'When did Caroline go to the LGBTQ support group?', 'answer': '7 May 2023', 'category': 2, 'evidence': ['D1:3']}, {'sample_id': 'conv-26', 'question_id': 'conv-26_q_1', 'question': 'When did Melanie paint a sunrise?', 'answer': '2022', 'category': 2, 'evidence': ['D1:12']}, {'sample_id': 'conv-26', 'question_id': 'conv-26_q_2', 'question': 'What fields would Caroline be likely to pursue in her educaton?', 'answer': 'Psychology, counseling certification', 'category': 3, 'evidence': ['D1:9', 'D1:11']}, {'sample_id': 'conv-26', 'question_id': 'conv-26_q_3', 'question': 'What did Caroline research?', 'answer': 'Adoption agencies', 'category': 1, 'evidence': ['D2:8']}, {'sample_id': 'conv-26', 'question_id': 'conv-26_q_4', 'question': "What is Caroline's identity?", 'answer': 'Transgender woman', 'category': 1, 'evidence': ['D1:5']}, {'sa

In [7]:
%pip install sentence_transformers

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
from difflib import SequenceMatcher
from sentence_transformers import SentenceTransformer, util

_model = SentenceTransformer('all-MiniLM-L6-v2')

def check_similarity_string(text1: str, text2: str) -> float:
    if not text1 or not text2:
        return 0.0
    return SequenceMatcher(None, text1.lower(), text2.lower()).ratio()


def check_similarity_semantic(text1: str, text2: str) -> float:
    if not text1 or not text2:
        return 0.0
    emb1 = _model.encode(text1, convert_to_tensor=True)
    emb2 = _model.encode(text2, convert_to_tensor=True)
    return float(util.cos_sim(emb1, emb2).item())

c:\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
import pandas as pd
import time
import requests

def ask(question):
    resp = requests.post("http://localhost:8003/ask", params={"question": question})
    return resp.json().get("answer")

qa = []

total_questions = len(questions.get("questions", []))

for idx, question in enumerate(questions.get("questions", []), start=1):
    print(f"[{idx}/{total_questions}] Asking question: {question.get('question')}")
    output = measure_function(ask, question.get("question"))
    print(f"[{idx}/{total_questions}] Answer:", output["result"])
    
    answer_received = output["result"]
    metrics = output["metrics"]
    
    similarity_string = check_similarity_string(question.get("answer"), answer_received)
    similarity_semantic = check_similarity_semantic(question.get("answer"), answer_received)
    
    row = {
        "memory": MEMORY_BACKEND,
        "time": time.time(),
        "conversation_index": conversation_index,
        "question": question.get("question"),
        "answer_actual": question.get("answer"),
        "answer_received": answer_received,
        "similarity_string": similarity_string,
        "similarity_semantic": similarity_semantic,
        "similarity": (similarity_string + similarity_semantic) / 2,
        **metrics,
        "toxic_latency": TOXIC_LATENCY,
        "toxic_bandwidth": TOXIC_BANDWIDTH,
        "toxic_slow_close": TOXIC_SLOW_CLOSE,
        "toxic_timeout": TOXIC_TIMEOUT,
        "toxic_slicer": TOXIC_SLICER,
        "toxic_limit_data": TOXIC_LIMIT_DATA,
        "toxic_reset_peer": TOXIC_RESET_PEER
    }
    
    print(row)
    qa.append(row)

Asking question: When did Caroline go to the LGBTQ support group?
Starting function 'ask' at 1762441988.1643536
Function 'ask' completed at 1762442056.652406
Answer: The memory doesn't specify when Caroline attended the LGBTQ support group.
{'memory': 'graphiti', 'time': 1762442059.1426446, 'conversation_index': 0, 'question': 'When did Caroline go to the LGBTQ support group?', 'answer_actual': '7 May 2023', 'answer_received': "The memory doesn't specify when Caroline attended the LGBTQ support group.", 'similarity_string': 0.09523809523809523, 'similarity_semantic': 0.07154262065887451, 'similarity': 0.08339035794848487, 'cpu_edge_ns': 2183776272669.1672, 'cpu_cloud_ns': 188761794895.60788, 'ram_edge_bytes': 46783989940.70588, 'ram_cloud_bytes': 53789776835.76462, 'disk_edge_bytes': 92335410106.0317, 'disk_cloud_bytes': 43371202.88038836, 'network_edge_bytes': 177268.12077716461, 'network_cloud_bytes': 298957.4576334998, 'openai_tokens': 358, 'openai_cost_usd': 0.0009603999999999999, 

In [ ]:
if qa:
    df = pd.DataFrame(qa)
    output_dir = "results"
    os.makedirs(output_dir, exist_ok=True)
    
    is_constrained = (
        int(TOXIC_LATENCY) > 0 or
        int(TOXIC_BANDWIDTH) > 0 or
        int(TOXIC_SLOW_CLOSE) > 0 or
        int(TOXIC_TIMEOUT) > 0 or
        int(TOXIC_SLICER) > 0 or
        int(TOXIC_LIMIT_DATA) > 0 or
        int(TOXIC_RESET_PEER) > 0
    )
    
    if is_constrained:
        output_file = f"{MEMORY_BACKEND}_conv_{conversation_index}_qa_constrained.csv"
    else:
        output_file = f"{MEMORY_BACKEND}_conv_{conversation_index}_qa.csv"
    
    output_path = os.path.join(output_dir, output_file)
    
    if os.path.exists(output_path):
        backup_path = os.path.join(output_dir, f"{MEMORY_BACKEND}_conv_{conversation_index}_qa_backup.csv")
        if is_constrained:
            backup_path = os.path.join(output_dir, f"{MEMORY_BACKEND}_conv_{conversation_index}_qa_constrained_backup.csv")
        shutil.move(output_path, backup_path)
        print(f"Existing file backed up to {backup_path}")
    
    df.to_csv(output_path, index=False)
    print(f"Saved new results to {output_path}")